In [1]:
import json
import torch
import numpy as np

## List of hyperparameters
* Learning rate
    * Momentum decay
    * or step learningrate
* Batch size - 32
    * Explained: The batch size is a hyperparameter that defines the number of samples to work through before updating the internal model parameters.
    
* Number of Epochs (include early stopping) - 100 step learning rate on [30, 50]
* Paths for:
    * training data 
    * test data 
    * Storing and restoring models 
    
* Optimizer
    * Adam
    * RMSprop
    * AdaGrad
    
* Model:
    * Config hidden layers 
        * hidden_channels = [256, 128, 64, 1]

    * Padding : 'same'  

    * kernal size 
        * constant or list[hiddenLayers]

    * dilation rate 
        * layer dependant?

    * initializa weights 
    * initialize bias

In [2]:
# The impact of using return_sequences is that the model will classify each frame in one category.

DATA_FORMAT        = 'channel_last'
PADDING            = 'same'
RETURN_SEQUENCE    = True
NUM_INPUT_VARS     = 4
OUTPUT_KERNEL_SIZE = 1
OUTPUT_FILTER      = 1

from sclouds.helpers import get_lon_array, get_lat_array

n_lon = len(get_lon_array())
n_lat = len(get_lat_array())

# -- Preparatory code -- #############
# Model configuration    #############
batch_size = 100
no_epochs = 30
learning_rate = 0.001
validation_split = 0.2
verbosity = 1
########################################


def build_model(hp):
    """" Building a ConvLSTM model for predicting cloud cover.
    All filters are squared. Adding the architecture.

    Parameteres
    ------------------------
    filters : array like
        use length of this to infer the depth of the network.


    Returns
    ------------------------
    model : tensorflow.keras.Sequential
        Builded model
    """
    # TODO : update with activations?
    # recurrent_activation='hard_sigmoid'
    #                   , activation='tanh'
    filters, kernels, seq_length = 24
    
    model =  Sequential()
    # Adding the first layer
    seq.add(ConvLSTM3D(filters = filters[0],
                       kernel_size = (kernels[0], kernels[0], NUM_INPUT_VARS),
                       input_shape = (seq_length, n_lat, n_lon, NUM_INPUT_VARS),
                       kernel_initializer='glorot_uniform',
                       padding = PADDING,
                       return_sequences=RETURN_SEQUENCE,
                       data_format=DATA_FORMAT))

    prev_filter = filters[0]
    for i, filter, kernal in enumerate(zip(filters[1:], kernels[1:])):
        # Begin with 3D convolutional LSTM layer
        seq.add(keras.layers.ConvLSTM3D(filters=filter,
                                        kernel_size=(kernal, kernel, prev_filter),
                                        kernel_initializer='glorot_uniform',
                                        padding = PADDING,
                                        return_sequences=RETURN_SEQUENCE,
                                        data_format=DATA_FORMAT))
        prev_filter = filter
    # Adding the last layer
    seq.add(keras.layers.ConvLSTM3D(filters=OUTPUT_FILTER,
                                    kernel_size=(1, 1, prev_filter),
                                    kernel_initializer='glorot_uniform',
                                    padding = PADDING,
                                    return_sequences=RETURN_SEQUENCE,
                                    data_format=DATA_FORMAT))

    return model


In [ ]:
filters=hp.Choice(
    'num_filters',
    values=[32, 64],
    default=64,
),

In [22]:
path = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/storedModel/'

In [23]:
with open(path + 'config.json', 'w') as config_file:
    json.dump(hp, config_file)

In [ ]:
tuner = RandomSearch(build_model,
                     objective='mean_squared_error',
                     max_trials=5,
                     executions_per_trial=3,
                     directory='my_dir',
                     project_name='helloworld')

In [ ]:
tuner.search(x, y,
             epochs=5,
             validation_data=(val_x, val_y))

In [ ]:
tuner.search_space_summary() # printing search space summary 

In [ ]:
tuner.results_summary()      # printing result summary

In [ ]:
models = tuner.get_best_models(num_models=2)

In [ ]:
def keras_custom_loss_function(y_actual, y_predict):
    """  """
    import numpy as np
    return np.square(np.subtract(y_true, y_pred)).sum(axis = 0)